In [1]:
import torch 
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint, quad
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
#Exercise 1.1

class LQRSolver:
    def __init__(self, H, M, C, R, D, T, sigma):
        self.H = H
        self.M = M
        self.C = C
        self.R = R
        self.D = D
        self.T = T
        self.sigma = np.eye(2) * sigma

    def riccati_ode(self, S_flat, t):
        S = S_flat.reshape(2, 2)
        SDot = -2 * self.H.T @ S - S @ self.M @ np.linalg.inv(self.D) @ self.M.T @ S + self.C
        return SDot.flatten()

    def solve_riccati(self, time_grid):
        S0 = self.R.flatten()
        S_sol = odeint(self.riccati_ode, S0, time_grid)
        return S_sol.reshape(-1, 2, 2)

    def integrate_trace_term(self, S_t, t0, t1):
        """Numerical integration of the trace term from t0 to t1."""

        def trace_integrand(r):
            index = int((r - t0) / self.dt)
            # Ensure we get a matrix for S, not a scalar
            S = S_t[index].reshape(2, 2)
            # Ensure self.sigma is a 2D array
            trace_val = np.trace(self.sigma @ self.sigma.T @ S)
            return trace_val

        # Set self.dt as the smallest interval in your time grid for accurate integration
        # This assumes time_grid is sorted and uniform
        self.dt = t1 - t0 if t1 > t0 else self.dt

        integral, _ = quad(trace_integrand, t0, t1)
        return integral

    def compute_value_function(self, t_batch, x_batch):
        # Ensure t_batch is a tensor
        if not isinstance(t_batch, torch.Tensor):
            t_batch = torch.tensor([t_batch], dtype=torch.float64)
        S_t = self.solve_riccati(t_batch.numpy())  # Assuming this returns a list/array of 2x2 matrices
        values = torch.zeros(t_batch.size(0), 1)
        for i, (t, S) in enumerate(zip(t_batch, S_t)):
            x = x_batch[i]
            # Ensure x is 2-dimensional
            if x.ndim == 1:
                x = x.unsqueeze(1)  # Convert a 1D array to a 2D column vector
            S = torch.tensor(S).float()
            # Perform matrix multiplication
            v = x.T @ S @ x
            # Add integration if not at the last timestep
            integral = 0  # Initialize integral
            if i < len(t_batch) - 1:
                next_t = float(t_batch[i + 1])
                integral = self.integrate_trace_term(S, next_t)
            values[i] = v.squeeze() + integral
        return values

    def compute_control_function(self, t_batch, x_batch):
        S_t = self.solve_riccati(t_batch)
        controls = torch.zeros(t_batch.size(0), 2, dtype=torch.float)
        D_tensor = torch.tensor(self.D, dtype=torch.float)
        M_tensor = torch.tensor(self.M, dtype=torch.float)  # Convert self.M to a tensor of the same type

        for i, (t, S) in enumerate(zip(t_batch, S_t)):
            x = x_batch[i].float()
            S_tensor = torch.tensor(S, dtype=torch.float)
            control = -torch.linalg.inv(D_tensor) @ M_tensor.T @ S_tensor @ x
            controls[i, :] = control.flatten()

        return controls

In [5]:
#Exercise 1.2

# Problem constants from provided parameters
H = torch.tensor([[1.0, 0.0], [0.0, 1.0]]) * 0.1
M = torch.tensor([[1.0, 0.0], [0.0, 1.0]])
sigma = 0.001
C = torch.tensor([[0.0, 0.0], [0.0, 0.0]])
D = torch.tensor([[1.0, 0.0], [0.0, 1.0]])
R = torch.tensor([[1.0, 0.0], [0.0, 1.0]]) * 10
T = 1.0  # Assuming final time T=1, this needs to be provided

# Initial state
x0 = torch.tensor([1.0, 1.0], dtype=torch.float64)  # Example initial state
lqr_solver = LQRSolver(H.numpy(), M.numpy(), C.numpy(), R.numpy(), D.numpy(), T, sigma)

def simulate_explicit_step(S, x, a, dt, sigma):
    dW = sigma * np.sqrt(dt) * np.random.randn(*x.shape)
    dx = (H @ x + M @ a) * dt + dW
    return x + dx

# Function to simulate one step of the system using the implicit scheme
def simulate_implicit_step(S, x, dt, sigma):
    num_samples, _ = x.shape
    identity_matrix = np.eye(D.shape[0])
    # This matrix A is the same for all samples since S, D, M are constant for all samples
    A = identity_matrix + dt * np.linalg.inv(D) @ M.T @ S
    # Pre-compute this part for all samples
    b_common = dt * np.linalg.inv(D) @ M.T @ S
    # Initialize the new control array
    a_new = np.empty_like(x)
    # Generate the random noise
    dW = sigma * np.sqrt(dt) * np.random.randn(num_samples, D.shape[0])
    # Solve the system for each sample
    for i in range(num_samples):
        # Compute the right-hand side of the equation
        b = b_common @ x[i]
        # Solve the linear system for the control a
        a_new[i] = np.linalg.solve(A, b)
        # Now compute the new state using the control a and the implicit scheme
        x[i] = x[i] + dt * (H @ x[i] + M @ a_new[i]) + dW[i]
    return x, a_new

def monte_carlo_simulation(lqr_solver, N, num_samples, x0, scheme='explicit'):
    dt = T / N
    x = np.tile(x0.numpy(), (num_samples, 1))  # Ensure x0 is an ndarray
    total_costs = np.zeros(num_samples)

    S_sol_cache = np.array(lqr_solver.solve_riccati(np.linspace(0, T, N+1)))

    v0 = lqr_solver.compute_value_function(torch.tensor(0), x0.unsqueeze(0)).numpy()
    if np.isscalar(v0):  # This checks if v0 is a scalar
        v0_array = np.array([v0])  # This converts the scalar to a 1D array
    else:
        v0_array = np.ravel(v0)  # This flattens the array if it's more than 1D

    for n in range(N):
        S = S_sol_cache[n]
        a = -np.linalg.inv(D.numpy()) @ M.numpy().T @ S @ x.T
        if scheme == 'explicit':
            x += simulate_explicit_step(S, x, a, dt, sigma)
        elif scheme == 'implicit':
            x, a = simulate_implicit_step(S, x, dt, sigma)
        
        # Update total costs
        total_costs += np.sum(x * (C.numpy() @ x.T).T, axis=1) * dt
        total_costs += np.sum(a * (D.numpy() @ a.T).T, axis=1) * dt

    # Calculate the final cost and add it to the total costs
    total_costs += np.sum(x * (R.numpy() @ x.T).T, axis=1)

    # Compute errors and MSE
    errors = total_costs - v0_array  # Now v0_array is guaranteed to be 1D
    mse = np.mean(errors**2)
    return mse

In [6]:
# Now we can perform the simulations and plot the results
time_steps = [1, 10, 50, 100, 500, 1000, 5000]  # Example time steps
num_samples = [10, 50, 100, 500, 1000, 5000]  # Example Monte Carlo sample sizes

mse_values_time_explicit = [monte_carlo_simulation(lqr_solver, n, 1000, x0, 'explicit') for n in time_steps]
mse_values_time_implicit = [monte_carlo_simulation(lqr_solver, n, 1000, x0, 'implicit') for n in time_steps]

# Log-Log plot for MSE against time steps
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.loglog(time_steps, mse_values_time_explicit, 'o-', label='Error vs Time Steps')
plt.xlabel('Number of Time Steps')
plt.ylabel('MSE')
plt.title('Log-Log Plot of MSE vs Time Steps')
plt.legend()

plt.subplot(1, 2, 1)
plt.loglog(time_steps, mse_values_time_implicit, 'o-', label='Error vs Time Steps')
plt.xlabel('Number of Time Steps')
plt.ylabel('MSE')
plt.title('Log-Log Plot of MSE vs Time Steps')
plt.legend()

plt.tight_layout()
plt.show()

ValueError: diff requires input that is at least one dimensional